In [1]:
!pip install pypdf faiss-cpu sentence-transformers groq

In [14]:
import os

os.environ["GROQ_API_KEY"] = "key"

In [15]:
import faiss
import numpy as np
from groq import Groq
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer


In [21]:
#load PDF

from pypdf import PdfReader

def load_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_text = load_pdf("Sample.pdf")


In [22]:
#chunk Text

def chunk_text(text, chunk_size=400, chunk_overlap=50):
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size

        # Prevent cutting words
        if end < text_length:
            end = text.rfind(" ", start, end)
            if end == -1:
                end = start + chunk_size

        chunk = text[start:end].strip()
        chunks.append(chunk)

        start = end - chunk_overlap
        if start < 0:
            start = 0

    return chunks

In [23]:
documents = chunk_text(pdf_text)
print("Total chunks:", len(documents))
print(documents[0])

Total chunks: 2
"Redmi Note 13 is priced at ₹17,999 and has a 4.4 star rating. It is considered the best value-for-
money smartphone under ₹20,000.", 
    "Samsung Galaxy F54 costs ₹19,499 with a 4.3 star rating. It offers good camera performance but is 
slightly expensive.", 
    "Realme Narzo 70 is priced at ₹18,299 and has a 4.2 star rating. It is good for daily use.", 
    "Among all phones under


In [24]:
#create embeddings

from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-MiniLM-L6-v2")

doc_embeddings = embedder.encode(
    documents,
    show_progress_bar=True,
    convert_to_numpy=True
)

print(doc_embeddings.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(2, 384)


In [30]:
import numpy as np

doc_embeddings = doc_embeddings / np.linalg.norm(
    doc_embeddings, axis=1, keepdims=True
)

In [31]:
#create FAISS vectorDB

dimension = doc_embeddings.shape[1]

index = faiss.IndexFlatIP(dimension)
index.add(np.array(doc_embeddings))

print("Vector DB size:", index.ntotal)


Vector DB size: 2


In [32]:
#Retrieval Function (Core of RAG)


def retrieve_context(query, top_k=3, threshold=0.4):
    # Encode + normalize query
    query_embedding = embedder.encode(
        [query],
        convert_to_numpy=True
    )
    query_embedding = query_embedding / np.linalg.norm(
        query_embedding, axis=1, keepdims=True
    )

    # FAISS search
    distances, indices = index.search(query_embedding, top_k)

    best_score = distances[0][0]

    # Relevance gate
    if best_score < threshold:
        return [], best_score

    retrieved_docs = [documents[i] for i in indices[0]]
    return retrieved_docs, best_score

In [33]:
#Create RAG Agent (LLM + Context)

client = Groq()

def rag_agent(question):
    context_docs, score = retrieve_context(question)

    print("🔍 Similarity score:", score)

    # No relevant context → stop
    if not context_docs:
        return "The information is not available in my data."

    print("🔍 Retrieved context:")
    for doc in context_docs:
        print("-", doc[:120])

    context_text = "\n".join(context_docs)

    system_prompt = (
        "You are a RAG-based AI agent. "
        "Answer ONLY using the provided context. "
        "If the answer is not present, say: "
        "'The information is not available in my data.'"
    )

    user_prompt = f"""
Context:
{context_text}

Question:
{question}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content



In [34]:
#testing valid question

print(rag_agent("Which is the best phone under ₹20,000?"))


🔍 Similarity score: 0.59222376
🔍 Retrieved context:
- good for daily use.", 
    "Among all phones under ₹20,000, Redmi Note 13 provides the best balance of price and rating.
- "Redmi Note 13 is priced at ₹17,999 and has a 4.4 star rating. It is considered the best value-for-
money smartphone und
- good for daily use.", 
    "Among all phones under ₹20,000, Redmi Note 13 provides the best balance of price and rating.
The Redmi Note 13 is considered the best phone under ₹20,000, providing the best balance of price and rating.


In [35]:
#testing invalid question

print(rag_agent("Does iPhone 15 support USB-C?"))


🔍 Similarity score: 0.22790474
The information is not available in my data.
